In [1]:
from textblob.classifiers import NaiveBayesClassifier
import numpy
import json
import nltk
import pickle

In [2]:
def load_obj(filename):
    with open(filename, 'rb') as handle:
        b = pickle.load(handle)
        return b
    
data = load_obj("with_tags.pickle")

In [3]:
include = []
do_not_include = []

sample = data.sample(10)

def categorize(to_include, vol_data):
    if to_include:
        include.append(vol_data)
    else:
        do_not_include.append(vol_data)

data.apply(lambda x: categorize(x["to_include"],x["json"]), axis=1)

5831    None
7952    None
4133    None
7231    None
6255    None
5769    None
3076    None
3420    None
860     None
5148    None
6924    None
5393    None
7663    None
2541    None
7583    None
4957    None
139     None
4310    None
5277    None
7041    None
1642    None
1444    None
3567    None
4397    None
4607    None
7578    None
7610    None
4817    None
4641    None
8213    None
        ... 
5394    None
4114    None
5637    None
4576    None
5667    None
7915    None
5925    None
8234    None
1621    None
3922    None
2210    None
104     None
821     None
5718    None
2191    None
2775    None
498     None
1389    None
8194    None
4255    None
946     None
7478    None
4662    None
6320    None
7848    None
5860    None
2743    None
5546    None
8077    None
7789    None
dtype: object

In [4]:
count_include = len(include)
count_do_not = len(do_not_include)

training_size_i = count_include*.8*.8
test_size_i1 = count_include*.8*.2
test_size_i2 = count_include*.2

training_size_d = count_do_not*.8
test_size_d1 = count_include*.8*.2
test_size_d2 = count_do_not*.2

test1 = []
test2 = []
train = []

i = 0
while i<training_size_i:
    train.append((include[i],"include"))
    i=i+1
while i<training_size_i+test_size_i1:
    test1.append((include[i],"include"))
    i=i+1
while i<count_include:
    test2.append((include[i],"include"))
    i=i+1
    
i = 0
while i<training_size_d:
    train.append((do_not_include[i],"do_not"))
    i=i+1
while i<training_size_d+test_size_d1:
    test1.append((do_not_include[i],"do_not"))
    i=i+1
while i<count_do_not:
    test2.append((do_not_include[i],"do_not"))
    i=i+1


In [5]:
print test1[0][0]
print test1[0][1]

{u'description': u"<p>One-shot story of Jackie Dio's transformation from small time enforcer to super powered villain.</p>", u'publisher': {u'api_detail_url': u'https://comicvine.gamespot.com/api/publisher/4010-31/', u'id': 31, u'name': u'Marvel'}, u'start_year': u'2006', u'first_issue': {u'api_detail_url': u'https://comicvine.gamespot.com/api/issue/4000-150573/', u'issue_number': u'1', u'id': 150573, u'name': u'Underworld (Part 1)'}, u'deck': None, u'image': {u'icon_url': u'https://comicvine.gamespot.com/api/image/square_avatar/685513-u1.jpg', u'medium_url': u'https://comicvine.gamespot.com/api/image/scale_medium/685513-u1.jpg', u'tiny_url': u'https://comicvine.gamespot.com/api/image/square_mini/685513-u1.jpg', u'small_url': u'https://comicvine.gamespot.com/api/image/scale_small/685513-u1.jpg', u'thumb_url': u'https://comicvine.gamespot.com/api/image/scale_avatar/685513-u1.jpg', u'screen_url': u'https://comicvine.gamespot.com/api/image/screen_medium/685513-u1.jpg', u'super_url': u'htt

In [6]:
from HTMLParser import HTMLParser
import string
from textblob.utils import strip_punc

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def corpus_from_text(data,ls,key):
    #print(data)
    if data[key] is not None:        
        desc = strip_tags(data[key]).lower()
        desc = desc.split()
        for word in desc:
            word = strip_non_ascii(word)
            word = strip_punc(word,all=True)
            #print(word)
            if word not in ls:
                ls.append(word)
    return ls

def strip_non_ascii(string):
    stripped = (c for c in string if (0 < ord(c) < 127))
    return ''.join(stripped)

def first_words(text,num):
    ls = ""
    j = 0
    if text is not None:
        desc = strip_tags(text).lower()
        desc = desc.split()
        for word in desc:
            word = strip_non_ascii(word)
            word = strip_punc(word,all=True)
            #print(word)
            ls = ls + word
            j = j+1
            if j == num:
                return ls
        return ls
    return ls

manual_desc_corpus = ['reprints','collection','collects','collected','collecting','hardcover','paperback','after','trade','as'
                     'into','vol']
manual_title_corpus = ['masterworks','death','without','knight','four','line','by','collection','omnibus']
desc_corpus = manual_desc_corpus
title_corpus = manual_title_corpus
                
def basic_word_extractor(data,key):
    corpus = desc_corpus
    if key == 'name':
        corpus = title_corpus
    data_words = []
    data_words = corpus_from_text(data,data_words,key)
    features = dict((((key+'_contains({0})').format(word), (word in data_words))
                                            for word in corpus))
    return features

def long_desc(desc):
    if desc is None:
        return False
    return len(strip_tags(desc))>=1000

def volume_features(vol_data):
    #words_in_title = words(vol_data['name'])
    #words_in_desc = words(vol_data['name'])
    features= {'issue_count': vol_data['count_of_issues'],
            'start_year': int(vol_data['start_year']),
            'early_start': int(vol_data['start_year'])<=1990,
            'first_word': first_words(vol_data['description'],1),
            'first_two': first_words(vol_data['description'],2),
            'long_desc': long_desc(vol_data['description']),
    }
    features.update(basic_word_extractor(vol_data,"name"))
    features.update(basic_word_extractor(vol_data,"description"))
    return features

In [7]:
train_feats = [(volume_features(n), category) for (n, category) in train]
test_feats = [(volume_features(n), category) for (n, category) in test1]
train_set = train_feats
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [8]:
accuracy1 = nltk.classify.accuracy(classifier, test_feats)
print(accuracy1)
classifier.show_most_informative_features(30)

0.970588235294
Most Informative Features
description_contains(reprints) = True           do_not : includ =     19.5 : 1.0
description_contains(paperback) = True           do_not : includ =     13.8 : 1.0
description_contains(trade) = True           do_not : includ =     11.1 : 1.0
              first_word = u'a'           do_not : includ =      8.4 : 1.0
description_contains(collecting) = True           do_not : includ =      7.5 : 1.0
             issue_count = 1              do_not : includ =      3.6 : 1.0
description_contains(collects) = True           do_not : includ =      3.2 : 1.0
              start_year = 2010           do_not : includ =      3.2 : 1.0
              start_year = 2009           do_not : includ =      3.1 : 1.0
              start_year = 2014           includ : do_not =      2.6 : 1.0
              start_year = 2015           includ : do_not =      2.6 : 1.0
              start_year = 1995           includ : do_not =      2.6 : 1.0
               first_two = ''

In [28]:
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs

def prob_of_label(feats,cl):
    arr = {}
    dist = cl.prob_classify(feats)
    for label in dist.samples():
        row = "%s: %f" % (label, dist.prob(label))
        #print(row)
        arr[label]=dist.prob(label)
    return arr

def trunc_desc(vol):
    if vol["description"] is not None:
        return vol["description"][0:30]
    else:
        return ""

errors = []
for (vol, tag) in test1:
    #print(vol['description'])
    if vol["name"] is not None:
        guess = classifier.classify(volume_features(vol))
        dist = prob_of_label(volume_features(vol),classifier)
        if guess != tag:
            errors.append( (tag, guess, vol["name"], vol['description'],vol['site_detail_url'], dist) )
        if ((dist['include']<.9) & (dist['include']>.1)):
            print (dist, tag, guess, vol["name"], vol['description'],vol['site_detail_url'])
        
i = 0
for item in errors:
    print(item)
    i = i+1
    
print i

({'include': 0.782247077031058, 'do_not': 0.21775292296894316}, 'include', 'include', u'Paradise X: Heralds', u'<p>Heralds takes place between the <a data-ref-id="4050-9149" href="/universe-x/4050-9149/" slug="universe-x">Universe X</a> and <a data-ref-id="4050-9547" href="/paradise-x/4050-9547/" slug="paradise-x">Paradise X</a> series.</p>', u'https://comicvine.gamespot.com/paradise-x-heralds/4050-19092/')
({'include': 0.8471828172623297, 'do_not': 0.15281718273766964}, 'include', 'include', u'All-New, All-Different Marvel Universe', u'<p>One shot guidebook.</p>', u'https://comicvine.gamespot.com/allnew-alldifferent-marvel-universe/4050-89255/')
({'include': 0.3973265699912982, 'do_not': 0.6026734300087022}, 'do_not', 'do_not', u'Very Best of Spider-Man', u'<p>.</p>', u'https://comicvine.gamespot.com/very-best-of-spider-man/4050-27331/')
('do_not', 'include', u'The Official Handbook of the Marvel Universe: Master Edition', u'<p>The Official Handbook of the Marvel Universe - Master Edi